In [1]:
import graphviz
import os
from datetime import datetime

def create_final_flowchart(output_path):
    """
    Erstellt das Flussdiagramm

    """
    
    dot = graphviz.Digraph(comment='Finaler Algorithmus-Flowchart')
    
    # --- Konfiguration ---
    dot.attr(rankdir='TB', dpi='300', splines='ortho')
    dot.attr('node', shape='box', style='filled', fillcolor='#F5F5F5', fontname='Arial', fontsize='11')
    dot.attr('edge', fontname='Arial', fontsize='10')
    dot.attr(label='Vereinfachtes Flussdiagramm des Threshold-Lookahead-Algorithmus', fontsize='16', fontname='Arial Bold', labelloc='t')

    # --- Legende für Abkürzungen (mit r"" für Raw String) ---
    with dot.subgraph(name='cluster_legend') as c:
        c.attr(label='Abkürzungen', fontname='Arial Bold', fontsize='12', color='grey')
        
        # KORREKTUR: r'''...''' verwendet, um SyntaxWarning zu vermeiden
        legend_text = r'''SoC: State of Charge (Ladezustand)
P_max: Max. Lade-/Entladeleistung
P[t]: Preis zum Zeitpunkt t
Q_low: Unterer Preisschwellenwert
Q_high: Oberer Preisschwellenwert
T_max: Gesamtdauer der Simulation
n: Größe des Zeitfensters (Anzahl Zeitschritte)'''
        # Hinweis: Bei Raw Strings mit '''...''' wird das letzte '\l' nicht benötigt.
        
        c.node('abbreviations', legend_text, shape='note', style='filled', fillcolor='#FFFFE0', align='left', fontsize='10')

    # --- Knoten des Flussdiagramms ---
    dot.node('start', 'Start', shape='ellipse', style='filled', fillcolor='#E0E0E0')
    dot.node('end', 'Ende', shape='ellipse', style='filled', fillcolor='#E0E0E0')
    dot.node('init', 'Initialisiere Parameter\n(SoC=0, P_max, t=0)')

    decision_style = {'shape': 'diamond', 'style': 'filled', 'fillcolor': '#DCDCDC'}
    dot.node('loop', 'Für jeden Zeitschritt t', **decision_style)
    dot.node('price_check', 'Prüfe Preis P[t]', **decision_style)
    dot.node('charge_check', 'P[t] < Q_low?', **decision_style)
    dot.node('capacity_check', 'SoC < Kapazität?', **decision_style)
    dot.node('discharge_check', 'P[t] > Q_high?', **decision_style)
    dot.node('soc_check', 'SoC > 0?', **decision_style)
    dot.node('end_check', 't ≥ T_max?', **decision_style)

    dot.node('window', 'Extrahiere Preisfenster\nP[t : t+n]')
    dot.node('quantiles', 'Berechne Schwellenwerte\nQ_low und Q_high')
    dot.node('update', 'Aktualisiere SoC, Zyklen & t')
    
    action_style = {'shape': 'box', 'style': 'filled,rounded', 'fillcolor': '#F5F5F5'}
    dot.node('charge', 'LADEN', **action_style)
    dot.node('discharge', 'ENTLADEN', **action_style)
    dot.node('wait', 'WARTEN', **action_style)

    # --- Verbindungen (mit xlabel statt label) ---
    dot.edge('start', 'init')
    dot.edge('init', 'loop')
    dot.edge('loop', 'window') 
    dot.edge('window', 'quantiles')
    dot.edge('quantiles', 'price_check')
    dot.edge('price_check', 'charge_check')
    
    # KORREKTUR: xlabel anstatt label
    dot.edge('charge_check', 'capacity_check', xlabel='Ja')
    dot.edge('capacity_check', 'charge', xlabel='Ja')
    dot.edge('capacity_check', 'wait', xlabel='Nein')
    dot.edge('charge_check', 'discharge_check', xlabel='Nein')
    dot.edge('discharge_check', 'soc_check', xlabel='Ja')
    dot.edge('soc_check', 'discharge', xlabel='Ja')
    dot.edge('soc_check', 'wait', xlabel='Nein')
    dot.edge('discharge_check', 'wait', xlabel='Nein')
    
    dot.edge('charge', 'update')
    dot.edge('discharge', 'update')
    dot.edge('wait', 'update')
    dot.edge('update', 'end_check')
    
    dot.edge('end_check', 'loop', xlabel='Nein')
    dot.edge('end_check', 'end', xlabel='Ja')
    
    # --- Speichern der Datei ---
    filename = 'algorithmus_final_korrigiert'
    filepath = os.path.join(output_path, filename)
    
    try:
        dot.render(filepath, format='pdf', view=False, cleanup=True)
        dot.render(filepath, format='png', view=False, cleanup=True)
        print(f"✓ Finale Version des Flussdiagramms erfolgreich erstellt:")
        print(f"  - {filepath}.pdf")
        print(f"  - {filepath}.png")
    except graphviz.backend.ExecutableNotFound as e:
        print(f"❌ FEHLER: Graphviz nicht gefunden. Bitte installiere es und füge es zum System-PATH hinzu.")
        print(f"   (Fehlermeldung: {e})")

    return dot

# --- Skript ausführen ---
if __name__ == "__main__":
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = f"bachelorarbeit_diagramme_{timestamp}"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    print(f"Diagramme werden im Ordner '{output_dir}' gespeichert.")
    print("-" * 50)

    create_final_flowchart(output_dir)
    
    print("-" * 50)
    print("✅ Skript erfolgreich und ohne Warnungen beendet.")

Diagramme werden im Ordner 'bachelorarbeit_diagramme_20250808_065854' gespeichert.
--------------------------------------------------
✓ Finale Version des Flussdiagramms erfolgreich erstellt:
  - bachelorarbeit_diagramme_20250808_065854\algorithmus_final_korrigiert.pdf
  - bachelorarbeit_diagramme_20250808_065854\algorithmus_final_korrigiert.png
--------------------------------------------------
✅ Skript erfolgreich und ohne Warnungen beendet.
